<div class="alert alert-block alert-warning">
<b>Disclaimer:</b> The main objective of the jupyter notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function
- setting and run the fit   
- extracting and displaying information about the results

These steps have a minimizer-dependent syntax. That's one of the reasons why different minimizers have been used in the notebooks provided as examples.
But, the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.
</div>

<h1> Example: Gaussian Model 3D fitted with lmfit </h1>

<a id='Table of Contents'></a><h1>Table of Contents</h1>
- <a href='#introduction'>Introduction</a>
- <a href='#imports'>Importing the required libraries</a>
- <a href='#anim_plot'>Plot of the fitting model</a>
- <a href='#ref_data'>Creating the reference data</a>
- <a href='#fitting'>Setting and fitting </a>  
- <a href='#plotting_results'>Plotting the results</a>  

(<a href='#Table of Contents'>Top</a>)<a id='introduction'></a><h2>Introduction</h2>

<div class="alert alert-block alert-info">
The objective of this notebook is to show how to use the <b>Gaussian Model 3D</b> model to perform some 
fits using <a href="https://lmfit.github.io/lmfit-py/">lmfit</a>.
</div>

### Physical units
For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.

The dictionary of units defined in the cell below specify the units of the refined parameters adapted to the convention used in the experimental datafile.

In [ ]:
# Units of parameters for selected QENS model and experimental data
dict_physical_units = {'omega': "meV", 
                       'q': "1/Angstrom", 
                       'D': "meV.Angstrom^2", 
                       'variance_ux': "Angstrom", 
                       'scale': "unit_of_signal.meV",
                       'center': "meV"}

(<a href='#Table of Contents'>Top</a>)<a id='imports'></a><h2>Importing the required librairies</h2>

In [ ]:
# install QENSmodels (if not already installed)
import pkgutil
import sys

if not pkgutil.find_loader("QENSmodels"):
    buttonY = pnw.Button(name='Yes', button_type='success')
    buttonN = pnw.Button(name='No', button_type='danger')
    choice_installation = panel.Column("Do you want to install the QENSmodels' library?", panel.Row(buttonY, buttonN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("QENSmodels"):
    if buttonY.clicks>0:
        !{sys.executable} -m pip install git+https://github.com/QENSlibrary/QENSmodels#egg=QENSmodels
    elif buttonN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
# import python modules for plotting, fitting
from __future__ import print_function
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# for interactive plot
from pandas import DataFrame
import panel.widgets as pnw
import panel as pn
pn.extension()

# import model from QENS library
import QENSmodels

In [ ]:
# install lmfit (if not already installed)
if not pkgutil.find_loader("lmfit"):
    lmfitY = pnw.Button(name='Yes', button_type='success')
    lmfitN = pnw.Button(name='No', button_type='danger')
    choice_installation = panel.Column("Do you want to install lmfit?", panel.Row(lmfitY, lmfitN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("lmfit"):
    if lmfitY.clicks>0:
         !{sys.executable} -m pip install lmfit
    elif lmfitN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
# required imports from lmfit
from lmfit import Model, Parameters

(<a href='#Table of Contents'>Top</a>)<a id='anim_plot'></a><h2>Plot of the fitting model</h2>

The widget below shows the peak shape function imported from QENSmodels where the function's parameters can be varied.

In [ ]:
def mplplot(df, **kwargs):
    fig = df.plot(legend=False).get_figure()
    plt.grid()
    plt.ylabel('Gaussian Model 3D')
    plt.xlabel('x')
    plt.close(fig)
    return fig

def gaussian_model_3d(q=1.0, scale=1.0, center=0.0, D=1., variance_ux=1.0, view_fn=mplplot):
    xs = np.linspace(-10,10,100)
    ys = QENSmodels.sqwGaussianModel3D(xs, q, scale, center, D, variance_ux) 
    df = DataFrame(dict(y=ys), index=xs)
    return view_fn(df, q=q, scale=scale, center=center, D=D, variance_ux=variance_ux)

q = pnw.FloatSlider(name='q', value=1, start=0.2, end=10)
scale  = pnw.FloatSlider(name='scale', value=5, start=1., end=10)
center = pnw.FloatSlider(name='center', value=5, start=0, end=10)
D = pnw.FloatSlider(name='D', value=1, start=0, end=10)
variance_ux =  pnw.FloatSlider(name='variance_ux', value=1, start=0.01, end=10)

def on_click(event):
    """Reset the interactive plots to inital values."""
    print("reset values")
    q.value = 1
    scale.value = 5
    center.value = 5
    D.value = 1
    variance_ux.value = 1
    
reset_button = pnw.Button(name='Reset')

widgets = pn.Column("#### Gaussian Model 3D", q, scale, center, D, variance_ux, reset_button)
gaus_model_3d_panel = pn.Row(gaussian_model_3d(q.value, scale.value, center.value, D.value, variance_ux.value), widgets)

def update(event):
    gaus_model_3d_panel[0] = gaussian_model_3d(q.value, scale.value, center.value, D.value, variance_ux.value)
    
q.param.watch(update, 'value')
scale.param.watch(update, 'value')
center.param.watch(update, 'value')
D.param.watch(update, 'value')
variance_ux.param.watch(update, 'value')


reset_button.param.watch(on_click, 'clicks')

gaus_model_3d_panel

(<a href='#Table of Contents'>Top</a>)<a id='ref_data'></a><h2>Creating the reference data</h2>

In [ ]:
xx = np.linspace(-2,2,200)
gaussian_model_3d_noisy = QENSmodels.sqwGaussianModel3D(xx, .5, 0.7, 0.3, 1, 1.2)*(1+0.1*np.random.normal(0,1,200)) + 0.01*np.random.normal(0,1,200)
# plot initial mode
plt.plot(xx,gaussian_model_3d_noisy)
plt.grid()
plt.show()

(<a href='#Table of Contents'>Top</a>)<a id='fitting'></a><h2>Setting and fitting</h2>

In [ ]:
gmodel = Model(QENSmodels.sqwGaussianModel3D)

print('Names of parameters:', gmodel.param_names)
print('Independent variable(s):', gmodel.independent_vars)

initial_parameters_values = [1.22, 0.25, .3, 0.33]

# Define boundaries for parameters to be refined
gmodel.set_param_hint('scale', min=0)
gmodel.set_param_hint('center', min=-5, max=5)
gmodel.set_param_hint('D', min=0)
gmodel.set_param_hint('variance_ux', min=0)

# Fix some of the parameters
gmodel.set_param_hint('q', vary=False)

# Fit
result = gmodel.fit(gaussian_model_3d_noisy, w=xx, q=1.,
                    scale=initial_parameters_values[0], 
                    center=initial_parameters_values[1],
                    D=initial_parameters_values[2],
                    variance_ux=initial_parameters_values[3])

In [ ]:
# Plot Initial model and reference data
plt.plot(xx, gaussian_model_3d_noisy, 'b-', label='reference data')
plt.plot(xx, result.init_fit, 'k--', label='model with initial guesses')
plt.xlabel('x')
plt.grid()
plt.title('Initial model and reference data')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.show()

(<a href='#Table of Contents'>Top</a>)<a id='plotting_results'></a><h2>Plotting results </h2>
using methods implemented in `lmfit`

In [ ]:
# display result
print('Result of fit:\n',result.fit_report())

# plot fitting results using lmfit functionality
result.plot()

In [ ]:
# plot fitting results and reference data using matplotlib.pyplot
plt.plot(xx, gaussian_model_3d_noisy, 'b-', label='reference data')
plt.plot(xx, result.best_fit, 'r.', label='fitting result')
plt.title('Fit result and reference data')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('x')
plt.grid()